<a href="https://colab.research.google.com/github/SaravanaKumarRM/Amazon_Scrape/blob/main/Amazon_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

In [ ]:
!pip install pandas

In [ ]:
!apt install chromium-chromedriver xvfb > /dev/null

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

import time

In [5]:
opt = webdriver.ChromeOptions()
opt.add_argument('--headless')
opt.add_argument('--no-sandbox')
opt.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=opt)

In [6]:
df = pd.read_csv('Amazon_Scraping_Sheet1.csv')
asins = df["Asin"]
countries = df["country"]
Product_list = []
web_404 = []

In [7]:
class Amazon:

    def title(self):
        product_title = driver.find_element(By.ID,"productTitle").text
        return product_title
    def price(self):
        try:
            product_price = driver.find_element(By.XPATH,".//span[@class='a-color-base']").text
        except NoSuchElementException:
            product_price = driver.find_element(By.XPATH,"/html/body/div[2]/div[2]/div[5]/div[4]/div[4]/div[11]/div[1]/div[1]/span[1]/span[2]").text
        except Exception as e:
            print(e)
        return product_price
    def image(self):
        image_container = driver.find_element(By.ID,"main-image-container")
        product_img = image_container.find_element(By.TAG_NAME,"img").get_attribute("src")
        return product_img
    def details(self):
        try:
            product_details = driver.find_element(By.ID,"detailBullets_feature_div").text
        except NoSuchElementException:
            product_details = driver.find_element(By.ID,"productDetails_techSpec_section_1").text
        except Exception as e:
            print(e)
        return product_details

In [ ]:
for country,asin in zip(countries,asins):
    try:
        web = f'https://www.amazon.{country}/dp/{asin}'
        # web = 'https://www.amazon.de/dp/000420736X'
        driver.get(web)
        website = Amazon()

        Product = {"Web_pageurl":web,"Product_Title":website.title(),"Product_price":website.price(),"Product_imagelink":website.image(),"Product_Details":website.details()}
        Product_list.append(Product)
        print(Product_list)

    except:
        web_404.append(web)
        print(f"The {web} is not avaliable")

In [9]:
file_json = pd.DataFrame(Product_list)
file_json.to_json("Amazon_scrape.json")

file_csv = pd.DataFrame(Product_list)
file_csv.to_csv("Amazon_scrape.csv",encoding='utf-8',index=False)

file_404 = pd.DataFrame(web_404)
file_404.to_csv("web_404.csv",encoding='utf-8',index=False)

driver.implicitly_wait(10)
driver.quit()